In [1]:
#Create S3 Bucket

In [16]:
import boto3
import sagemaker

session = boto3.session.Session()
region = session.region_name
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

s3 = boto3.Session().client(service_name="s3", region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [17]:
try:
    print("--- Checking for sagemaker_session ---")
    print(sagemaker_session)
    
    print("\n--- Checking for bucket ---")
    print(bucket)
    
    print("\n✅ Both variables appear to be defined.")

except NameError as e:
    print(f"\n❌ An error occurred: {e}")
    print("It looks like one of the variables is not defined. Please re-run the setup cell that defines them.")

--- Checking for sagemaker_session ---

--- Checking for bucket ---
sagemaker-us-east-1-564543410445

✅ Both variables appear to be defined.


In [19]:
# Define a clear project prefix for the organization
project_prefix = "crime-prediction-datalake"

# Define the S3 path for our "raw zone" data lake
raw_data_prefix = f"{project_prefix}/data/raw"

# The local path to your raw dataset
local_raw_data_path = "crime_data_10k_rows.csv" 

# Upload the raw data to the S3 data lake
s3_uri_raw_data = sagemaker_session.upload_data(
    path=local_raw_data_path,
    bucket=bucket,
    key_prefix=raw_data_prefix
)

print(f"✅ Successfully uploaded raw data to your S3 Data Lake at: {s3_uri_raw_data}")

✅ Successfully uploaded raw data to your S3 Data Lake at: s3://sagemaker-us-east-1-564543410445/crime-prediction-datalake/data/raw/crime_data_10k_rows.csv


In [20]:
import time
import pandas as pd

# --- Configuration ---
# The database and table names created with the Glue Crawler
DATABASE_NAME = "crime_data_db"
# The table name is  the name of the source file
TABLE_NAME = "raw" 

# --- Create an Athena Client ---
athena_client = boto3.client("athena", region_name=region)

# --- Define S3 path for query results ---
# Athena needs a place in S3 to store the results of any query.
query_results_prefix = f"{project_prefix}/query-results/"
s3_results_path = f"s3://{bucket}/{query_results_prefix}"

# --- Create and Run the Query ---
print("Running Athena query...")
query = f'SELECT * FROM "{DATABASE_NAME}"."{TABLE_NAME}" LIMIT 1000;'

response = athena_client.start_query_execution(
    QueryString=query,
    ResultConfiguration={"OutputLocation": s3_results_path},
)

query_execution_id = response["QueryExecutionId"]

# --- Wait for the Query to Finish ---
state = "RUNNING"
while state in ["RUNNING", "QUEUED"]:
    response = athena_client.get_query_execution(QueryExecutionId=query_execution_id)
    state = response["QueryExecution"]["Status"]["State"]
    if state == "FAILED":
        print("Query FAILED!")
        print(response["QueryExecution"]["Status"]["StateChangeReason"])
        break
    if state == "SUCCEEDED":
        print("Query SUCCEEDED!")
        break
    time.sleep(2)

# --- Load Results into a Pandas DataFrame ---
if state == "SUCCEEDED":
    s3_output_location = response["QueryExecution"]["ResultConfiguration"]["OutputLocation"]
    
    # The result file has a .csv extension, but its name is the query execution ID
    results_filename = f"{query_execution_id}.csv"
    s3_results_file_path = f"{s3_output_location.replace(s3_results_path, '')}{results_filename}"

    # Use pandas to read the CSV results directly from S3
    df_from_athena = pd.read_csv(s3_output_location)
    
    print(f"\nSuccessfully loaded query results into a pandas DataFrame.")
    print(f"DataFrame shape: {df_from_athena.shape}")
    display(df_from_athena.head())

Running Athena query...
Query SUCCEEDED!

Successfully loaded query results into a pandas DataFrame.
DataFrame shape: (1000, 28)


,dr_no,date rptd,date occ,time occ,area,area name,rpt dist no,part 1-2,crm cd,crm cd desc,...,status,status desc,crm cd 1,crm cd 2,crm cd 3,crm cd 4,location,cross street,lat,lon
0,190326475,03/01/2020 12:00:00 AM,03/01/2020 12:00:00 AM,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,AA,Adult Arrest,510.0,998.0,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506
1,200106753,02/09/2020 12:00:00 AM,02/08/2020 12:00:00 AM,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,998.0,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628
2,200320258,11/11/2020 12:00:00 AM,11/04/2020 12:00:00 AM,1700,3,Southwest,356,1,480,BIKE - STOLEN,...,NaN,NaN,NaN,NaN,480.0,NaN,NaN,NaN,NaN,NaN
3,200907217,05/10/2023 12:00:00 AM,03/10/2020 12:00:00 AM,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,IC,Invest Cont,343.0,NaN,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387
4,200412582,09/09/2020 12:00:00 AM,09/09/2020 12:00:00 AM,630,4,Hollenbeck,413,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,200 E AVENUE 28,NaN,34.0820,-118.2130


In [21]:
# Display summary statistics for numerical columns
print(df_from_athena.describe())

# Check data types and non-null counts
print(df_from_athena.info())

# See the distribution of a specific categorical column
# (You'll need to replace 'your_column_name' with a real one from your data)
# print(df_from_athena['your_column_name'].value_counts())

              dr_no    time occ         area  rpt dist no     part 1-2  \
count  1.000000e+03  1000.00000  1000.000000  1000.000000  1000.000000   
mean   2.006627e+08  1350.74800     6.612000   705.592000     1.396000   
std    4.952254e+05   650.32085     3.718236   369.675341     0.489309   
min    1.903265e+08     1.00000     1.000000   101.000000     1.000000   
25%    2.003179e+08   915.00000     3.000000   392.750000     1.000000   
50%    2.006185e+08  1430.00000     6.000000   668.000000     1.000000   
75%    2.010097e+08  1900.00000    10.000000  1021.000000     2.000000   
max    2.013099e+08  2355.00000    13.000000  1395.000000     2.000000   

            crm cd     vict age   premis cd  weapon used cd    crm cd 1  \
count  1000.000000   827.000000  814.000000      224.000000  571.000000   
mean    511.540000    36.587666  287.124079      378.566964  475.854641   
std     200.450265    74.749593  215.606152      122.713984  180.283748   
min     121.000000     0.000000  